# SLT-CE-2: Deterministic Annealing

Legi 16-352-137



### References

<ol>
<li> Deterministic annealing for clustering, compression, classification, regression, and related optimization
problems, Kenneth Rose, 1998, http://ieeexplore.ieee.org/document/726788/
</li>
    
<li>
A Ratio Scale Metric and the Compatibility
of Ratio Scales: The Possibility of
Arrow’s Impossibility Theorem, T.L. Saalty, 1994, https://www.sciencedirect.com/science/article/pii/0893965994900930
</li>

<li>
The wine data set, http://www3.dsi.uminho.pt/pcortez/wine5.pdf
</li>
    
<li>
Lecture 4, slide 19, https://ml2.inf.ethz.ch/courses/slt/lectures/slt20_lecture04.pdf
</li>
    
</ol>

### Setup 

In [1]:
import sklearn as skl
import sklearn.cluster as cluster
import sklearn.model_selection as model_selection
import sklearn.svm as svm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Make sure to install treelib in the slt-ce conda environment: conda install treelib
import treelib as tl

from sklearn.utils.validation import check_is_fitted


<h2 style="background-color:#f0b375;">
Section Preliminary
<span style=font-size:50%> Complete all problems in this and previous sections to get a grade of 0.0 </span>
</h2>

<p style="background-color:#adebad;">
    Implement the function read_X_y_from_csv according to the contract in its docstring.
</p>

In [26]:
def read_X_y_from_csv(sheet, y_names=None):
    """Parse a column data store into X, y arrays

    Args:
        sheet (str): Path to csv data sheet.
        y_names (list of str): List of column names used as labels.

    Returns:
        X (np.ndarray): Array with feature values from columns that are not contained in y_names (n_samples, n_features)
        y (dict of np.ndarray): Dictionary with keys y_names, each key contains an array (n_samples, 1)
                                with the label data from the corresponding column in sheet. 
    """

    # Your code goes here
    df = pd.read_csv(sheet)
    y = dict({})
    for name in y_names:
        y[name] = df[name].values
        df = df.drop(name, axis=1)
    X = df.values
    return X, y

<p style="background-color:#adebad;">
Read the wine data [3], which contains 11 physiochemical attributes, and two labels (quality and color).
</p>

In [32]:
X, y = read_X_y_from_csv("wine-data.csv", y_names=["quality", "color"])

<h2 style="background-color:#f0b375;">
Section 4.0
<span style=font-size:50%> Complete all problems in this and previous sections to get a grade of 4.0 </span>
</h2>

<p style="background-color:#adebad;">
    Read reference [1] about deterministic annealing clustering (DAC). Shortly summarize what they refer to as the <i>preferred implementation</i> of the DAC algorithm.
</p>

<i> Preferred implementation </i> is a mass-constrained version of deterministic annealing which peculiarity is in its independance on the initialization of clusters-codevectors. We need only to choose maximal number of clusters possible to appear, and then through decreasing the temperature  the algorithm  will increase number of clusters until minimum temperature is achieved. At every step all obtained clusters are checked for splitting, and if they split, new are added (algorithm procedure in detail is on the page 9 of [1]).

<p style="background-color:#adebad;">
    Implement the <b>fit method</b> for the template class DeterministicAnnealing, according to the contract outlined in its docstring.
    You can add more class methods as necessary.
    See http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html for complementary information.
</p>

In [85]:
print(np.shape(np.mean(X, axis=0)))
a = np.empty((1,11))
a[0,:] = np.mean(X, axis=0)
print(a)
print(np.shape(a))
print(np.mean(X, axis=0))
np.sum(np.ones((np.shape(X)[1],1)), axis=1)
np.shape(X)
np.cov(X)

(11,)
[[7.21282900e+00 3.37145606e-01 3.16607665e-01 5.44209635e+00
  5.37195629e-02 3.05122364e+01 1.15741188e+02 9.92387831e-01
  3.21612590e+00 5.28910266e-01 1.04898923e+01]]
(1, 11)
[7.21282900e+00 3.37145606e-01 3.16607665e-01 5.44209635e+00
 5.37195629e-02 3.05122364e+01 1.15741188e+02 9.92387831e-01
 3.21612590e+00 5.28910266e-01 1.04898923e+01]


array([[2550.33477645, 1930.23954174, 1446.4602644 , ...,  615.72010606,
         615.72010606,  615.72010606],
       [1930.23954174, 1517.22724889, 1097.58655237, ...,  462.89754306,
         462.89754306,  462.89754306],
       [1446.4602644 , 1097.58655237,  828.04498576, ...,  357.25380042,
         357.25380042,  357.25380042],
       ...,
       [ 615.72010606,  462.89754306,  357.25380042, ...,  159.11137506,
         159.11137506,  159.11137506],
       [ 615.72010606,  462.89754306,  357.25380042, ...,  159.11137506,
         159.11137506,  159.11137506],
       [ 615.72010606,  462.89754306,  357.25380042, ...,  159.11137506,
         159.11137506,  159.11137506]])

In [100]:
class DeterministicAnnealingClustering(skl.base.BaseEstimator, skl.base.TransformerMixin):
    """Template class for DAC
    
    Attributes:
        cluster_centers_ (np.ndarray): Cluster centroids y_i (n_clusters, n_features)
        cluster_probabs_ (np.ndarray): Assignment probability vectors p(y_i | x) for each sample
                                       (n_samples, n_clusters)
        bifurcation_tree_ (treelib.Tree): Tree object that contains information about cluster evolution during
                                          annealing.
                                       
    Parameters:
        n_clusters (int): Maximum number of clusters returned by DAC.
        random_state (int): Random seed.
    """
    
    def __init__(self, n_clusters=8, random_state=42, metric="euclidian",\
                 Tmin = 0.1, alpha=0.95):
        self.n_clusters = n_clusters
        self.random_state = random_state
        self.metric = metric
        # Add more parameters, if necessary.
        self.current_n_clusters = 1
        self.cluster_probabs_ = None
        self.cluster_centers_ = None
        self.bifurcation_tree_ = None
        self.y_prob = None # Marginal probabilities of clusters
        self.Tmin = Tmin
        self.alpha = alpha
        
    def fit(self, X):
        """Compute DAC for input vectors X
        
        Preferred implementation of DAC as described in reference [1].
        Consider to use initialization and reseeding as in sklearn k-means for improved performance.
        
        Args:
            X (np.ndarray): Input array with shape (samples, n_features)
        
        Returns:
            self
        """
        
        Tmin_flag = False
        converged_flag = False
        N = np.shape(X)[0] # Number of samples
        F = np.shape(X)[1] # Number of features
        while not (Tmin_flag):
            if self.current_n_clusters == 1:
                C_x = np.cov(X)
                eig = np.linalg.eigvals(C_x)
                ab_eig = np.absolute(eig)
                lambda_max = np.max(ab_eig)
                lambda_max = np.max(np.abs(np.linalg.eigvals(C_x)))
                T = 3*lambda_max
                self.cluster_centers_ = np.empty((1,F))
                self.cluster_centers_[0, :] = np.mean(X, axis=0)
                self.cluster_probabs_ = np.ones((F,1))
                self.y_prob = np.sum(self.cluster_probabs_, axis=0)
                
            converged_flag = False    
            while not converged_flag:
                if self.metric == "euclidian":

                    # Your code goes here

                    # update p(y|x)
                    for i in range(self.current_n_clusters):
                        self.cluster_probabs_[i, :] = self.y_prob[i] * \
                            np.exp(-1.0/T*np.linalg.norm(X - self.cluster_centers_[i, :])**2)
                    for i in range(self.current_n_clusters): 
                        self.cluster_probabs_[:, i] /= sum(self.cluster_probabs_[:, i]) 

                    # update p(y)    
                    self.y_prob = np.sum(self.cluster_probabs_, axis=0) / N

                    # update y
                    cluster_centers_new = X @ self.cluster_probabs_ / self.y_prob / N
                    
                    # Convergence check
                    if np.linalg.norm(cluster_centers_new - self.cluster_centers_) < 1e-5:
                        converged_flag = True
                    self.cluster_centers_ = cluster_centers_new
                        
                #elif metric == "ratioscale":
                    # code for extension
                    
                # Temperature check and cooling step
                if T <= self.Tmin:
                    T = 1e-9
                    Tmin_flag = True
                else:
                    T *= self.alpha
                print(T)
                
                if self.current_n_clusters < self.n_clusters:
                    for i in range(self.current_n_clusters):
                        # Calculate covariance matrix of posterior
                        C = np.zeros(F)
                        for j in range(N):
                            D = (X[j,:] - self.cluster_centers_[i, :]) \
                            @ np.transpose(X[j,:] - self.cluster_centers_[i, :])
                            C += 1.0 / N * self.cluster_probabs_[j, i] * D
                        
                        # Add cluster depending on covariance and temperature
                        l_max = max(np.abs(np.linalg.eigvals(np.cov(C))))
                        if T < 2*l_max:
                            print(self.current_n_clusters)
                            self.current_n_clusters += 1
                            self.cluster_centers_ = np.vstack(self.cluster_centers_,\
                                                             self.cluster_centers_[i,:])
                            a = np.random.rand(np.shape(self.cluster_centers_[0,:]))
                            self.cluster_centers[self.current_n_clusters-1, :] += a
                            self.y_prob[:, i] /= 2.0
                            self.y_prob = np.hstack(self.y_prob, self.y_prob[:, i])                            

        return self
    
    def predict(self, X):
        """Predict assignment probability vectors for each sample in X.
        
        Args:
            X (np.ndarray): Input array with shape (new_samples, n_features)
            
        Returns:
            P (np.ndarray): Assignment probability vectors (new_samples, n_clusters) 
        """
        
        # Your code goes here
        
        
        return P
    
    def transform(self, X):
        """Transform X to a cluster-distance space.
        
        In the new space, each dimension is the distance to the cluster centers. 
        
        Args:
            X (np.ndarray): Input array with shape (new_samples, n_features)
            
        Returns:
            Y (np.ndarray): Cluster-distance vectors (new_samples, n_clusters)
        """
        check_is_fitted(self, ["cluster_centers_"])
        
        # Your code goes here
        
        
        
        return Y
    
    def plot_bifurcation(self):
        """Show the evolution of cluster splitting"""
        check_is_fitted(self, ["bifurcation_tree_"])
        
        # Your code goes here
        
        return None
        

<p style="background-color:#adebad;">
    Create an instance of your DAC class with n_clusters = 2 and <b>fit the first 6000 samples</b> of the wine data set. Record the execution time. Furthermore, create an instance of the sklearn k-means class, and fit it with the same parameters. Again record the execution time.
</p>

In [53]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,
                                                                        y["color"],
                                                                        train_size=6000,
                                                                        random_state=42)

/home/anton/anaconda3/envs/slt-ce/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2069: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [ ]:
%%time
DAC = DeterministicAnnealingClustering(n_clusters=2, random_state=42)
DAC.fit(X_train)

In [60]:
%%time
kmeans = cluster.KMeans(n_clusters=2,random_state=42)
kmeans.fit(X_train)

CPU times: user 235 ms, sys: 7.97 ms, total: 243 ms
Wall time: 199 ms


<h2 style="background-color:#f0b375;">
Section 4.5
<span style=font-size:50%> Complete all problems in this and previous sections to get a grade of 4.5 </span>
</h2>

<p style="background-color:#adebad;">
    <ul style="background-color:#adebad;">
        <li> 
            Implement the <b>predict method</b> for the template class DAC, according to the contract outlined in its docstring.
        </li>
        <li>
            Use DAC.predict and kmeans.predict to predict the cluster labels of X_test.
        </li>
        <li>
            Compute the confusion matrix between the two predictions as described in <br>
            http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
        </li>
    </ul>
</p>

In [ ]:
%%time
y_kmeans = kmeans.predict(X_test)

In [ ]:
%%time
y_DAC = DAC.predict(X_test)

<ul style="background-color:#adebad;">
<li> Before we can compute the confusion matrix, we need to perform some post-processing on the DAC cluster assignments.
    Explain what the function postprocess (defined below) does, and why we need it. To do so, complete the docstring of the function postprocess.
        </li>
</ul>

In [58]:
def postprocess(y_DAC, y_kmeans):
    """TODO: Add explanation"""
    
    y_DAC_hard = np.argmax(y_DAC, axis=1)
    
    n_clusters = len(np.unique(y_DAC_hard))
    dac2kmeans = []
    for cluster in range(n_clusters):
        argmax = np.argmax(y_DAC[:, cluster])
        dac2kmeans.append(y_kmeans[argmax])
        
    y_DAC_new = []
    for dac_label in y_DAC_hard:
        y_DAC_new.append(dac2kmeans[dac_label])
        
    return np.array(y_DAC_new)

In [ ]:
sklearn.metrics.confusion_matrix(y_kmeans, postprocess(y_DAC, y_kmeans))

<h2 style="background-color:#f0b375;">
Section 5.0
<span style=font-size:50%> Complete all problems in this and previous sections to get a grade of 5.0 </span>
</h2>

<ul style="background-color:#adebad;">
        <li> Implement the <b>transform method</b> for the template class DAC, according to the contract outlined in its docstring.
        </li>
        <li>
        Use DAC.transform and kmeans.transform to transform both, X_train and X_test. 
        </li>
       
</ul>

In [ ]:
X_train_DAC = DAC.transform(X_train)
X_test_DAC = DAC.transform(X_test)

X_train_kmeans = kmeans.transform(X_train)
X_test_kmeans = kmeans.transform(X_test)

<ul style="background-color:#adebad;">
        <li>
        Fit an SVM classifier with default parameters to the untransformed data, and to the transformed data.
        Compare the performance of predicting whether the color of a wine is red or white.
        </li>
    </ul>

In [ ]:
svm = svm.SVC(random_state=42)
svm.fit(X_train, y_train)
svm.score(X_test, y_test)

In [ ]:
svm_DAC = svm.SVC(random_state=42)
svm_DAC.fit(X_train_DAC, y_train)
svm_DAC.score(X_test_DAC, y_test)

In [ ]:
svm = svm.SVC(random_state=42)
svm.fit(X_train_kmeans, y_train)
svm.score(X_test_kmeans, y_test)

<ul style="background-color:#adebad;">
        <li>
        Produce two scatter plots, one for X_train_DAC and one for X_train_kmeans.<br>
        Make the marker color indicate the wine color.
        </li>
    </ul>

<ul style="background-color:#adebad;">
    <li>
        Create a fixed 2D embedding (e.g. with LLE, t-SNE, MDS) of the wine data and color the markers according to quality and color. Fit and transform X_train with DAC(n_clusters=3,4,5,6,7,8,...). Produce a plot of the SVM score svm_DAC.score(X_test_DAC, y_test) as a function of n_clusters.. Each time use marker shapes to display the cluster memberships, and compare to the labels color and quality.
    </li>
</ul>

In [ ]:
"""
    %%time
    lle = skl.manifold.LocallyLinearEmbedding(random_state=...)
    lle.fit(...)
"""

<h2 style="background-color:#f0b375;">
Section 5.5
<span style=font-size:50%> Complete all problems in this and previous sections to get a grade of 5.5 </span>
</h2>
<ul style="background-color:#adebad;">
        <li>
            Produce a phase diagram plot of the expected distortion D, as shown in figure 2 of reference [1]. For this, extend DAC.fit to save the expected distortion during annealing as an additional attribute self.distortion.
            You might also want to save the number of effective clusters and the temperature along the way.
        </li>
    </ul>
</p>

In [ ]:
# extend DAC.fit(self, X):
    # ...
    # Save information for each (n-th) annealing step:
    # self.distortion = [d0, d1, d2, ...]
    # self.n_eff_clusters = [e0, e1, e2, ...]
    # self.temp = [t0, t1, t2, ...]
    # ...

<ul style="background-color:#adebad;">
        <li>
        Implement DAC.plot_bifurcation, which should create a bifurcation plot as shown on slide 19 of lecture 3. As our data is not 1-dimensional as in the lecture slide, we will have to adapt our scheme, so that the distances between nodes of the tree make sense.<br>
        Modify DAC.fit to keep track of the distances, using the tree object DAC.bifurcation\_tree\_. When a cluster splits, it creates two child nodes. Each node should store its centroid vector, and the distance to the parent centroid vector. After splitting, the parent node is not updated anymore.<br>
        In the bifurcation plot, the horizontal distance of a child node to its parent node should be exactly the distance to the parent centroid vector. The two child nodes should move in opposite directions, i.e. one to the left of the parent and one to the right.
        </li>
    </ul>

<ul style="background-color:#adebad;">
        <li>
        Argue how reasonable our method of plotting the bifurcation is. Explain how the 1D-distances between nodes (i.e. nodes that are not siblings) do not correspond exactly to the distances between centroids. Suggest ideas for improvement.
        </li>
    </ul>

<h2 style="background-color:#f0b375;">
Section 6.0
<span style=font-size:50%> Complete all problems in this and previous sections to get a grade of 6.0 </span>
</h2>

<ul style="background-color:#adebad;">
        <li>
            So far, our implementation of DAC assumed that our data is compatible with the euclidian metric. Argue why this assumption is not justified for the wine-data.
        </li>
    </ul>
</p>



<ul style="background-color:#adebad;">
    <li>
        All the features of the wine-data set are measured on a ratio scale, which is incompatible with the euclidian metric (Remark: this is not the complete answer to problem 6, argue why they are not compatible). A more appropriate distance is proposed in reference [2]:
            <br><br>
            $d(x,y)=\log{ \frac{1}{d^2} \sum_{i,j=1}^d \frac{x_i}{x_j} \frac{y_j}{y_i}}$
            <br><br>
            Extend DAC.fit to the case of metric == ratioscale, using d(x,y) as given above.<br>
            Hint 1: As this distance does not give a closed form update formula for the centroids $y$, you will need to do gradient descent to update the centroids. You can either calculate the gradient by hand, or use an automatic differentiation tool like Tensorflow. If you calculate the gradient by hand, provide the formula in Latex below.
                    <br>
            Hint 2: Keep in mind the possibility of negative definite matrices and appropriate regularization when solving for the critical temperature of the ratio scale approach.
        </li>



<ul style="background-color:#adebad;">
    <li>
    Perform experiments to compare the euclidian and ratioscale metrics.
    </li>
</ul>

<h2 style="background-color:#4286f4;"> Comments </h2>

Let us know what you liked about this exercise, and what we can improve!